In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os.path
from glob import glob
import bs4
import urllib

In [19]:
base = 'https://chicago.councilmatic.org/'
suffix = '/search/?selected_facets=topics_exact%3AZoning+Reclassification'
page_query = '&page={num}'
resp = urllib.request.urlopen(base + suffix + page_query.format(num=1))
soup = bs4.BeautifulSoup(resp.read(), "html5lib")

In [140]:
# go to ordinance page to get additional info
# inner_resp = urllib.request.urlopen(base + ord_info['href'])
inner_resp = urllib.request.urlopen(base + 'legislation/o2018-8121')

inner_soup = bs4.BeautifulSoup(inner_resp.read(), "html5lib")




[('Osterman, Harry', 'Ward 48'),
 ('Moore, Joseph', 'Ward 49'),
 ('Mitchell, Gregory I.', 'Ward 7'),
 ('Smith, Michele', 'Ward 43'),
 ('Scott, Jr. Michael', 'Ward 24'),
 ('Ervin, Jason C.', 'Ward 28'),
 ('Dowell, Pat', 'Ward 3'),
 ('Tunney, Thomas', 'Ward 44'),
 ('Pawar, Ameya', 'Ward 47'),
 ('King, Sophia D.', ''),
 ('Cappleman, James', 'Ward 46'),
 ('Sadlowski Garza, Susan', 'Ward 10'),
 ('Mell, Deborah', 'Ward 33'),
 ('Maldonado, Roberto', 'Ward 26'),
 ('Reboyras, Ariel', 'Ward 30'),
 ('Santiago, Milagros', ''),
 ('Waguespack, Scott', 'Ward 32'),
 ('Taliaferro, Chris', 'Ward 29'),
 ('Moreno, Proco Joe', 'Ward 1'),
 ('Arena, John', 'Ward 45'),
 ('Burnett, Jr., Walter', 'Ward 27'),
 ('Thompson, Patrick D.', 'Ward 11')]

In [119]:
ord_info

{'ord': 'O2018-8387',
 'href': '/legislation/o2018-8387/',
 'sponsor': 'Laurino, Margaret'}

In [ ]:
def get_ords(soup):
    '''get each ordinance & its link'''
    ord_info = {}
    for link in soup.find_all('a', class_='small'):
        ord_info = {'ord': link.contents[0].split()[1],
                'href': link.get('href')}
    return ord_info



def get_locs(inner_soup, ord_info):
    '''get locations mentioned in ordinance'''
    locs = []
    for inner_link in inner_soup.find_all('ul', class_='list-unstyled'):
        if str(inner_link.previous_element.previous_element).strip() == 'Locations mentioned':
            for list_item in inner_link.find_all('li'):
                locs.append(list_item.string.strip())
    ord_info['locations'] = locs
    return ord_info['locations']


def get_sponsors(inner_soup):
    sponsors = []
    wards = []
    inner_link = inner_soup.find('div', class_='table-responsive')
    spo = 1
    war = 1
    for col in inner_link.find_all('td'):
        if col.find('a'):
            sponsors.append(col.a.string.strip())
            spo += 1
        if col.string:
            wards.append(col.string.strip())
            war +=1
return sponsors,wards

In [16]:
ord_list = []
page_num = 0
while True:
    page_num = page_num+1
    try:
        resp = urllib.request.urlopen(base + suffix + page_query.format(num=page_num))
        soup = bs4.BeautifulSoup(resp.read(), "html5lib")
        
#         get ord info & link
        ord_info = get_ords(soup)
        # go to ordinance page to get additional info
        inner_resp = urllib.request.urlopen(base + ord_info['href'])
        inner_soup = bs4.BeautifulSoup(inner_resp.read(), "html5lib")
#         get location
        ord_info['locations'] = get_locs(inner_soup, ord_info)
        sponsors, spo_wards = get_sponsors(inner_soup)
        ord_info['sponsors'] = sponsors
        ord_info['sponsor_ward'] = spo_wards
        
        
#         append dict to list
        ord_list.append(ord_info)
    except:
        break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
